# General remarks and introduction

This document serves to demonstrate the simulation functions of the `sbmfi` (simulation based metabolic flux inference) `python` package that I have developed over the course of my PhD. The folder structure of the package is shown below:
```
sbmfi
|___ core
|___ inference
|___ models
|___ lcmsanalysis
```
The `core` folder contains all the algorithms that are necessary to specify a simulator that takes as input fluxes and outputs mass distribution vectors (MDV) of metabolites in a steady-state $^{13}C$ carbon labelling experiment (CLE) using the [EMU algorithm](https://www.sciencedirect.com/science/article/pii/S109671760600084X?via%3Dihub). It also contains a dataset simulator to simulate measurement. The `inference` folder contains a uniform prior over the flux polytope. The `models` folder is self-explanatory. Last, the `lcmsanalysis` contains the `formula` file that specifies a convenient class for chemical formulas.


### Constructing a simulator: `sbmfi.core`

The simulator of `sbmfi` inherits its design from the popular [`cobrapy` package](https://cobrapy.readthedocs.io/en/latest/). It is highly recommended to first familiarize one-self with `cobrapy` before working with this simulator.

All the computations for `sbmfi` are performed by one of two possible backends: `numpy` and `torch`. The reason for doing this is that `numpy` and the `scipy` extension are the standard scientific numerics packages in python. It is possible to use everything in `sbmfi.core` without having `torch` as a dependency, but for all functionality in the `sbmfi.inference` module, `torch` is a hard requirement.

The different backends are specified in `sbmfi.core.linalg.LinAlg`. Linalg has the following functions:
- setting a global seed for random number generation
- specifying a device where all tensors are stored (e.g. for use of `CUDA` on GPU), though GPU support has not been extensively tested 
- setting of `kwargs` of functions
- setting the `batch_size` parameter, which is the number of fluxes that are processed in batch during simulations

In [1]:
import pandas as pd
import torch
import numpy as np

In [2]:
from sbmfi.core.linalg import LinAlg

np_la = LinAlg(backend='numpy', seed=1, batch_size=2)  
tr_la = LinAlg(backend='torch', seed=1, batch_size=2)

a = np_la.get_tensor(shape=(3,3))
print(a, 'a numpy array')

b = tr_la.get_tensor(shape=(3,3))
print(b, 'a torch tensor')

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] a numpy array
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]) a torch tensor


### Constucting a model from string input

There are different ways of constructing a model. Shown below is the construction of a model completely from string inputs. 

It is also possible is to augment an [SBML model](https://sbml.org/) with atom-transition mappings. This way, we can use models from the [BiGG repository](http://bigg.ucsd.edu/models), which can be read by the `cobra.io` package. This way of model construction is used for the *E. coli* model that I use throughout my thesis. This method of model construction is discussed in the section on the Antoniewicz model below.

In [3]:
from sbmfi.core.model import EMU_Model

reaction_kwargs = {
    'a_in': {
        'upper_bound': 10.0, 
        'lower_bound': 10.0,
        'atom_map_str': '∅ --> A/abc'
    },
    'co2_out': {
        'upper_bound': 100.0,
        'atom_map_str': 'co2/a --> ∅'
    },
    'e_out': {
        'upper_bound': 100.0,
        'atom_map_str': 'E/ab --> ∅'
    },
    'v1': {
        'upper_bound': 100.0,
        'atom_map_str': 'A/abc --> B/ab + D/c'
    },
    'v2': {
        'upper_bound': 100.0,
        'atom_map_str': 'A/abc --> C/bc + D/a'
    },
    'v3': {
        'upper_bound': 100.0,
        'atom_map_str': 'B/ab + D/c --> E/ac + co2/b'
    },
    'v4': {
        'upper_bound': 100.0,
        'atom_map_str': 'C/ab + D/c --> E/cb + co2/a'
    }, 
}  
metabolite_kwargs = {
    'E': {'formula': 'C2H4O2'},
}

model = EMU_Model(id_or_model='bimodal', linalg=tr_la)

model.add_reactions(
    reaction_kwargs=reaction_kwargs,
    metabolite_kwargs=metabolite_kwargs
)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-09


Now we set the $^{13}C$-labelling of the substrate metabolite, which in this case is metabolite `A`. The convention is that 0s indicate a $^{12}C$ atom and 1s indicate a $^{13}C$ atom

In [4]:
input_labelling = pd.Series({f'A/011': 1.0,}, name='input')
model.set_input_labelling(input_labelling)

Last, we set the metabolite(-fragments) that we can measure given our experimental capabilities. This is important information for the EMU algorithm.

In [5]:
model.set_measurements(measurement_list=['E']) 

As in `cobra`, our model has reaction and metabolite objects, though these have been augmented in order to handle labelling simulations:

In [6]:
for reaction in model.reactions:
    print(reaction, reaction.bounds, type(reaction))

a_in:  --> A/abc (10.0, 10.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
co2_out: co2/a -->  (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
e_out: E/ab -->  (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v1: A/abc --> B/ab + D/c (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v2: A/abc --> C/bc + D/a (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v3: B/ab + D/c --> E/ac + co2/b (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v4: C/ab + D/c --> E/cb + co2/a (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>


In [7]:
for metabolite in model.metabolites:
    print(metabolite, type(metabolite))

A <class 'sbmfi.core.metabolite.EMU_Metabolite'>
co2 <class 'sbmfi.core.metabolite.EMU_Metabolite'>
E <class 'sbmfi.core.metabolite.EMU_Metabolite'>
B <class 'sbmfi.core.metabolite.EMU_Metabolite'>
D <class 'sbmfi.core.metabolite.EMU_Metabolite'>
C <class 'sbmfi.core.metabolite.EMU_Metabolite'>


In the cell below, we build the simulator. There are a couple of very important arguments in this function:
- `kernel_basis` $\in$ {`rref`, `svd`}: kernel of the null-space of the equality constraint matrix of the flux polytope
- `basis_coordinates` $\in$ {`rounded`, `transformed`}: whether the input to the simulator is in transformed or rounded coordinates
- `free_reaction_id` specifies which fluxes are considered free when using the row reduced echelon form (RREF) kernel basis and transformed coordinates
- `logit_xch_fluxes`: whether the input to the simulator has logit transformed exchange flux coordinates

In [8]:
model.build_simulator(
    free_reaction_id=['v4'],
    kernel_basis='rref',
    basis_coordinates='transformed',
    logit_xch_fluxes=False,
    verbose=False,
)

fcm = model.flux_coordinate_mapper
type(fcm)

sbmfi.core.polytopia.FluxCoordinateMapper

The `fcm` variable above is where fluxes are mapped between different coordinate systems. The simulator takes as input labelling fluxes, whereas the thermodynamic coordinate system is more convenient and interpretable and is what is typically reported in literature on fluxes. The relation between labelling and thermodynamic fluxes is discussed in the section on the Antoniewicz model. Sampling a polytope uniformely is done in a rounded full-dimensional polytope, which is again a different coordinate system for the same variables. When building our simulator, it has a `sbmfi.core.polytopia.FluxCoordinateMapper` in the back where all the relevant coordinate mappings are specified. For the `model` built above, we chose to use a `rref` null-space kernel and `transformed` coordinates, which corresponds to using free fluxes as our input coordinate system. 

`fcm` has three `sbmfi.core.polytopia.LabellingPolytope` attributes: `fcm._Fn`, `fcm._F` and `fcm._Ft` which represent the net-flux, labelling-flux and thermodynamic flux polytopes respectively. Below we see that the `net_flux_polytope` object has attributes `S` and `h` which represent the stoichiometric matrix and the equality constraints and attributes `A` and `b` which represent the inequality constraints.

In [9]:
net_flux_polytope = fcm._Fn
print(net_flux_polytope.__dict__.keys())
pd.concat([net_flux_polytope.S, net_flux_polytope.h], axis=1)  # last column is thus the equality constraints!

dict_keys(['A', 'b', 'shift', 'transformation', 'inequality_only', 'S', 'h', '_mapper', '_objective', '_cvx_result', '_nlr'])


,v1,v2,v3,co2_out,e_out,v4,a_in,eq
A,-1.0,-1.0,0.0,0.0,0.0,0.0,1.0,0
co2,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,0
E,0.0,0.0,1.0,0.0,-1.0,1.0,0.0,0
B,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0
D,1.0,1.0,-1.0,0.0,0.0,-1.0,0.0,0
C,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,0


Net fluxes are sampled from a affinely transformed and rounded polytope that is constructed in a `sbmfi.core.polytopia.PolytopeSamplingModel` object. This object is stored as the `fcm._sampler` attribute. The rounded polytope from which variables are sampled is stored in the `fcm._sampler._F_round` attribute, which only contains inequality constraints. 

In [10]:
rounded_net_flux_polytope = fcm._sampler._F_round
pd.concat([rounded_net_flux_polytope.A, rounded_net_flux_polytope.b], axis=1)  # last column is thus the inequality constraints!

,B_v4,ineq
v3|lb,5.0,5.0
v4|lb,-5.0,5.0


Our model only has a single free flux which we set to be `v4` and thus if we pick some values for 'v4' we can map it to fluxes. To figure out what values of `v4` are allowed, we can run Flux Variability Analysis (FVA):

In [11]:
from sbmfi.core.polytopia import fast_FVA
fast_FVA(fcm._Fn)

,min,max
v1,-0.0,10.0
v2,-0.0,10.0
v3,-0.0,10.0
co2_out,10.0,10.0
e_out,10.0,10.0
v4,-0.0,10.0
a_in,10.0,10.0


We chose `v4` as our free flux, and we see from the FVA results above that its range is `0.0 - 10.0`. Below we manually pick 4 free fluxes within this range:

In [12]:
free_fluxes = pd.DataFrame([4.0, 6.0, 5.0, 8.0], columns=['v4'])
free_fluxes

,v4
0,4.0
1,6.0
2,5.0
3,8.0


Many of the functions in `sbmfi` take the `pandalize=bool` kwarg which turns the output of that function into a `pd.DataFrame` with sensible axis labels. Below we map the free fluxes to labelling fluxes via the coordinate mapper:

In [13]:
fluxes = fcm.map_theta_2_fluxes(free_fluxes, pandalize=True)
fluxes

,v1,v2,v3,co2_out,e_out,v4,a_in
0,6.0,4.0,6.0,10.0,10.0,4.0,10.0
1,4.0,6.0,4.0,10.0,10.0,6.0,10.0
2,5.0,5.0,5.0,10.0,10.0,5.0,10.0
3,2.0,8.0,2.0,10.0,10.0,8.0,10.0


To now simulate mass distribution vectors for the measured metabolites (which in this case is only metabolite `E`), we set the labelling fluxes and then call `model.cascade()` to run the simulation. Note that we MUST set only 2 fluxes at a time, since that is the `batch_size` that we set in the `LinAlg` object above!

In [14]:
model.set_fluxes(fluxes.loc[[0, 1]])
mdvs_1 = model.cascade(pandalize=True).copy()
model.set_fluxes(fluxes.loc[[2, 3]])
mdvs_2 = model.cascade(pandalize=True).copy()
pd.concat([mdvs_1, mdvs_2])

mdv_id,E+0,E+1,E+2
0,0.24,0.52,0.24
1,0.24,0.52,0.24
2,0.25,0.50,0.25
3,0.16,0.68,0.16


In the cell above we see that for flux vecors `[0, 1]`, the MDVs are exactly the same, thus highlighting the issue of unidentifiability!

The fundamental calculation of the EMU simulation algorithm is the solution of a cascade of linear systems: $\pmb{X}_w = \pmb{A}_w^{-1}\cdot \pmb{B}_w \cdot \pmb{Y}_w$, where the subscript $w$ indicates the size of the EMUs in that step of the cascade. For more information on the algorithm, please read [the publication](https://www.sciencedirect.com/science/article/pii/S109671760600084X?via%3Dihub). We can inspect the matrices that are used for the simulation by calling `model.pretty_cascade(weight=...)`, which returns a `dict` with all the matrices of that size. Note since `batch_size=2`, we have one set of matrices per parsed flux-vector (in this case vectors `[2, 3]`)!

In [15]:
EMU_matrices = model.pretty_cascade(2)
print(EMU_matrices.keys())
EMU_matrices['B']

dict_keys(['A', 'B', 'X', 'Y'])


,,C|[1] ∗ D|[0],B|[0] ∗ D|[0]
2,"E|[0,1]",-5.0,-5.0
3,"E|[0,1]",-8.0,-2.0


# The Antoniewicz *E. coli* model

The organism on which most flux analyses have been performed in literature is *E. coli*. The most popular model for its metabolism and atom transitions was postulated in [Leighty et al. (2013)](https://linkinghub.elsevier.com/retrieve/pii/S1096717613000840) and was also adopted by the people in Julich for their models which are formulated as `.fml` files. The Leighty model exactly matches the [ecoli_model_level_3](https://github.com/modsim/FluxML/blob/master/examples/models/ecoli_model_level_3.fml) model. In the publication, they used six different singly-labelled glucose substrates to perform $^{13}$C-MFA.

I parsed the Leighty model into a BiGG model with corresponding reaction and metabolite identifiers and which can thus be combined with `.sbml` models from the [BiGG repository](http://bigg.ucsd.edu/). Below, I will briefly walk through the steps of how the model was derived. The section on simulating data is more relevant to the user, but for completeness, I think it is good to first talk about the construction of the model and it is an intuitive way to introduce the types of inputs to the simulator. 

### Model construction

In [16]:
import pandas as pd
import torch
import numpy as np
import pprint

from sbmfi.models.build_models import build_e_coli_anton_glc

`build_e_coli_anton_glc` is built to conveniently parse and combine all the inputs necessary for the simulator for the Antoniwicz model. Internally, we read the `sbmfi\models\sbml\e_coli_anton.xml` file (which is in `.sbml` format) and then add all of the atom mappings and metabolite kwargs similar as in the small model of the previous section. These simulator inputs are stored in the kwargs, whereas the `anton_model` is a `EMU_Model`

In [17]:
anton_model, anton_kwargs = build_e_coli_anton_glc(
    backend='torch',
    auto_diff=False,
    build_simulator=True,
    batch_size=2,
    seed=1,
)

the `kwargs['anton_reaction_kwargs']` is a nested dictionary that contains mapping between the models. The first level of the dictionary are pathways, the second level are strings `v1` to `v71`, which are the reaction names used by Leighty. For every reaction, there is a dictionary with mapping arguments. The `fluxml_id` is the name of the reaction in the `.fml` model and the `anton_str` is the atom mapping defined by Leighty. Many reactions in the Leighty model map onto multiple reactions in BiGG models, for instance reaction `v7` maps onto reactions `ENO, PGK` and `PGM` (shown below). Therefore, we manually constructed atom mapping strings for every BiGG reaction. The coeff argument is there in order to parse fluxes, since reactions that are defined in a reverse direction w.r.t. the BiGG reaction should be multiplied by -1.

In [18]:
ark = anton_kwargs['anton_reaction_kwargs']
pprint.pprint(f'Pathways: {ark.keys()}')

print(f"\n reaction v7:")
pprint.pprint(ark['Glycolysis']['v7'])

("Pathways: dict_keys(['Glycolysis', 'Pentose Phosphate Pathway', "
 "'Entner-Doudoroff Pathway', 'TCA Cycle', 'Glyoxylate Shunt', 'Amphibolic "
 "Reactions', 'Acetic Acid Formation', 'Amino Acid Biosynthesis', 'One-Carbon "
 "Metabolism', 'Oxidative Phosphorylation', 'Transhydrogenation', 'ATP "
 "Hydrolysis', 'Transport', 'Biomass Formation', 'CO2 Exchange', 'EXTRA', "
 "'MODEL_SOURCE'])")

 reaction v7:
{'anton_str': '3PG (abc) <=> PEP (abc)',
 'bigg_kwargs': {'ENO': {'atom_map_str': '2pg_c/abc <=> pep_c/abc'},
                 'PGK': {'atom_map_str': '3pg_c/abc <=> 13dpg_c/abc',
                         'coeff': -1},
                 'PGM': {'atom_map_str': '2pg_c/abc <=> 3pg_c/abc',
                         'coeff': -1}},
 'fluxml_id': 'EMP7_v7'}


In the Leighty model, linear amino acid biosynthesis pathways combine many (sometimes more than 10) reactions into one big reaction for the pathway. Since I do not know the atom mappings for every reaction in the pathway, I chose to adopt this clumping as well. To do so, I identified all reactions that constitute the pathway, e.g. `ASAD,..., METS`, and combined these into one large reaction with an associated `atom_map_str`. For example, methionine synthesis is shown below and I dubbed it the `met_syn` reaction, which is thus not found in the BiGG model repository. With GC-MS, one could only measure amino acid fragments, but in the LC-MS world, we are actually wasting quite some information by clumping stuff together. With LC-MS it might actually be possible to measure the intermediates of the `ASAD,..., METS` reactions and use them for inference, especially when the measurement of the resulting amino acid is difficult. Unclumping amino acid biosynthesis reactions could be useful if you can measure amino-acid biosynthesis pathway intermediates.

In [19]:
ark['Amino Acid Biosynthesis']['v51'] 

{'fluxml_id': 'AA15_v51___1|AA15_v51___2',
 'anton_str': 'Asp (abcd) + METHF (e) + Cys (fgh) + SucCoA (ijkl) + ATP + 2 NADPH --> Met (abcde) + Pyr (fgh) + Suc (0.5 ijkl + 0.5 lkji) + NH3',
 'bigg_kwargs': {'met_syn': {'to_combine': {'ASPK': {},
    'ASAD': {'coeff': -1},
    'HSDy': {'coeff': -1},
    'HSST': {},
    'SHSL1': {},
    'CYSTL': {},
    'METS': {},
    'METAT': {'coeff': 0},
    'HCYSMT': {'coeff': 0},
    'HCYSMT2': {'coeff': 0}},
   'atom_map_str': 'asp__L_c/abcd + 5mthf_c/e + cys__L_c/fgh + succoa_c/ijkl --> met__L_c/abcde + pyr_c/fgh + succ_c/ijkl'}}}

Since we use the formula of `EMU_Metabolite` objects to construct natural abundance correction matrices, and these take ages to compute for large metabolites, we set the formulas of large metabolites to only involve the moeieties that are involved in the labelling dynamics. Also, we need to signal the simulator that `succ, 26dap_LL` and `fum` have a rotational symmetry of $180^\circ$ (other rotational symmetries are not allowed and not typically necessary).

In [20]:
anton_kwargs['metabolite_kwargs']

{'succ': {'symmetric': True},
 'fum': {'symmetric': True},
 '26dap_LL': {'symmetric': True},
 'accoa': {'formula': 'C2H3O'},
 'succoa': {'formula': 'C4H6O4'},
 'methf': {'formula': 'C1'},
 '10fthf': {'formula': 'C1'},
 '5mthf': {'formula': 'C1'},
 'mlthf': {'formula': 'C1'}}

Leighty uses GC-MS as dependent variables for flux-inference, which are defined in the `1-s2.0-S1096717613000840-mmc1.pdf` document that is included in this repository. I mapped all those measurements to BiGG metabolites or metabolite fragments:

In [21]:
pprint.pprint({k:v for i, (k, v) in enumerate(anton_kwargs['anton_measurements'].items()) if i < 3})

{'Ala232': {'anton_pos': '2-3',
            'bigg_id': 'ala__L_c|[1,2]',
            'formula': 'C10H26ONSi2'},
 'Ala260': {'anton_pos': '1-2-3',
            'bigg_id': 'ala__L_c',
            'formula': 'C11H26O2NSi2'},
 'Gly218': {'anton_pos': '2', 'bigg_id': 'gly_c|[1]', 'formula': 'C9H24ONSi2'}}


We have parsed these measurements into an **`annotation_df` which is a central object in the simulation of measurements**. The `annotation_df` specifies which annotated features from some analytical chemistry method map onto what elements of the latent state of the simulator (the result of `model.cascade` shown above). The `annotation_df` is an input to the simulator and needs to be determined **by you!**. The `annotation_df` has four required columns: `met_id, nC13, formula` and `adduct_name`. In all mass-spectrometry based analytical chemistry (NMR is not included in `sbmfi`), ions can have different ionization states. The adducts that are common to LC-MS are specified in the `emzed_adducts` dataframe below. The parsed `annotation_df` for the Leighty model is also shown below. 

`annotation_df` required columns:
- `met_id`: identifier that must map onto a metabolite in `model.measurements`.
- `adduct_name`: adduct identifier tht must be present in the `emzed_adducts` dataframe
- `nC13`: annotated number of $^{13}$C carbons that this measurement has
- `formula`: formula of the measurement; the only requirement is that the number of carbons in this formula matches the number of carbons in the formula of a measured model metabolite `metabolite ∈ model.measurements`

The function `sbmfi.models.build_models._parse_anton_measurements` is used to do the parsing and the parsed result is stored in `parsed_anton_measurements.csv` in the models folder (which is included in this git repo, since some the parsing steps combined take a while). For quick access, the function `sbmfi.models.build_models.read_anton_measurements` reads `parsed_anton_measurements.csv` and extracts the `annotation_df` and `measurements` which are discussed below.

In [22]:
from sbmfi.lcmsanalysis.adducts import emzed_adducts
# adducts common to LC-MS
emzed_adducts.iloc[:5, :]

,m_multiplier,adduct_add,adduct_sub,z
adduct_name,,,,
M+,1,,,1
M+H,1,H,,1
M+NH4,1,NH4,,1
M+H+NH4,1,HNH4,,2
M+Na,1,Na,,1


In [23]:
# annotation_df for the Crown model
annotation_df = anton_kwargs['annotation_df']
annotation_df.iloc[:9, :]

,met_id,nC13,adduct_name,formula,sigma
0,"ala__L_c|[1,2]",0,M-,C10H26ONSi2,0.01
1,"ala__L_c|[1,2]",1,M-,C10H26ONSi2,0.01
2,"ala__L_c|[1,2]",2,M-,C10H26ONSi2,0.01
3,ala__L_c,0,M-,C11H26O2NSi2,0.01
4,ala__L_c,1,M-,C11H26O2NSi2,0.01
5,ala__L_c,2,M-,C11H26O2NSi2,0.01
6,ala__L_c,3,M-,C11H26O2NSi2,0.01
7,gly_c|[1],0,M-,C9H24ONSi2,0.01
8,gly_c|[1],1,M-,C9H24ONSi2,0.01


As a follow up to the Leighty publication, there was the [Crown et al. (2015)](https://www.sciencedirect.com/science/article/pii/S1096717615000038?via%3Dihub) publication, which extended the number of glucose labellings to 14 and performed MLE inference over all of them at once, resulting in probably the most accurate MLE estimate of *E. coli* fluxes ever done which is called the `COMPLETE-MFA` estimate. I have parsed all these labelling conditions (including carbon purity and natural $^{13}$C abundance) into a **`substrate_df` which is another important input to the simulator.**

The `substrate_df` requires unique index names and they signify the labelling experiment. Second, the columns of the `substrate_df` consist of the fractional abundances of isotopomers in the substrate mixture (including `co_2`). For instance for a 1-$^{13}$C labelled glucose experiment, denoted `[1]Glc` in the index, the relevant isotopomers are:
- `glc__D_e/100000 = 0.996000`
- `glc__D_e/000000 = 0.004000`
- `co2_e/0 = 0.989`
- `co2_e/1 = 0.011`

Note that the small abundance of `glc__D_e/000000` is due to the impurity of the $^{13}$C atoms, which is reported by the manufacturer of the substrate. In `sbmfi`, we assume that the impurity is randomly incorporated and thus that for substrates with multiple labelled atoms, the impurity is equally likely to occur at each labelled position. 

The function `sbmfi.models.build_models._parse_anton_substrates` is used to do the parsing and the parsed result is stored in `parsed_anton_substrates.csv`. For quick access, the function `sbmfi.models.build_models.read_anton_substrates` reads the file and extracts the `substrate_df`.

Below, we show the substrate purities reported in the Crown and Leighty publications:

In [24]:
anton_kwargs['anton_substrate_purity']

{'[]Glc': 0.0107,
 '[1,2]Glc': 0.995,
 '[2,3]Glc': 0.995,
 '[4,5,6]Glc': 0.999,
 '[2,3,4,5,6]Glc': 0.985,
 '[U]Glc': 0.984,
 '[1]Glc': 0.996,
 '[2]Glc': 0.995,
 '[3]Glc': 0.995,
 '[4]Glc': 0.992,
 '[5]Glc': 0.99,
 '[6]Glc': 0.985}

In [25]:
# susbtrate_df parsed from the Crown & Leighty publications
substrate_df = anton_kwargs['substrate_df']
sdf = substrate_df.iloc[-4:,:]
sdf.loc[:, ~(sdf == 0.0).all(0)]

,glc__D_e/000000,glc__D_e/000001,glc__D_e/001000,glc__D_e/000100,glc__D_e/000010,co2_e/0,co2_e/1
[3]Glc,0.005,0.000,0.995,0.000,0.00,0.989,0.011
[4]Glc,0.008,0.000,0.000,0.992,0.00,0.989,0.011
[5]Glc,0.010,0.000,0.000,0.000,0.99,0.989,0.011
[6]Glc,0.015,0.985,0.000,0.000,0.00,0.989,0.011


The measurements that are reported in the Crown publication are a combination of measurements from the 6 substrate labellings used in the Leighty publication and 8 new substrate labellings. All of the labellings are stored in the `1-s2.0-S1096717615000038-mmc5.xlsx` file that is included in the `sbmfi.models` folder. We parse these measurements by solving the following regression problem:
	
\begin{align}
\arg \min_{x_m^o} & \sqrt{ \mathbf{N}_m \cdot \vec{x}_m^o - \vec{x}_m^a}
\\
\text{s.t.} &
\\
& \vec{x} \geq 0.0 
\\
& \mathbf{1} \cdot \mathbf{N}_m \cdot \vec{x}_m^o = 1.0
\end{align}

Where $\mathbf{N}_m$ is the natural abundance correction matrix for metabolite $m$ (computed from the formulas in the `annotation_df` some cells above). $\vec{x}_m^a$ is the parsed measurement from the `1-s2.0-S1096717615000038-mmc5.xlsx` file, $\vec{x}_m^o$ are the measurements in our model. This computation ensures that the resulting $\vec{x}_m^o$ is non-negative, sums to 1 and is corrcted for natural abundance. Using $\mathbf{N}_m^{-1}$ as in [Nanchen et al. (2007)](https://link.springer.com/protocol/10.1007/978-1-59745-244-1_11) resulted in many negative values in $\vec{x}_m^o$, which I am not sure how to interpret (negative mass spec signals????).

The resulting `measurements` are then a `pd.Series` object with a `pd.MultiIndex`, where the first level indicates the labelling experiment, and the second level denotes the `data_id`:

In [26]:
measurements = anton_kwargs['measurements']

measurements.to_frame().T # all measurements across 14 labelling conditions

labelling_id              [1,2]Glc                        \
data_id      ala__L_c|[1,2]_{M-}+0 ala__L_c|[1,2]_{M-}+1   
0                         0.528513              0.081713   

labelling_id                                                        \
data_id      ala__L_c|[1,2]_{M-}+2 ala__L_c_{M-}+0 ala__L_c_{M-}+1   
0                         0.389773        0.526563        0.058002   

labelling_id                                                   \
data_id      ala__L_c_{M-}+2 ala__L_c_{M-}+3 gly_c|[1]_{M-}+0   
0                   0.401728        0.013707         0.603296   

labelling_id                                             ...          [6]Glc  \
data_id      gly_c|[1]_{M-}+1 val__L_c|[1,2,3,4]_{M-}+0  ... glu__L_c_{M-}+2   
0                    0.396704                  0.291249  ...        0.302949   

labelling_id                                                  \
data_id      glu__L_c_{M-}+3 glu__L_c_{M-}+4 glu__L_c_{M-}+5   
0                   0.043669        0.004671        0.002099   

labelling_id                                              \
data_id      tyr__L_c|[0,1]_{M-}+0 tyr__L_c|[0,1]_{M-}+1   
0                         0.968661              0.027614   

labelling_id                               BOM                        
data_id      tyr__L_c|[0,1]_{M-}+2 biomass_rxn EX_glc__D_e   EX_ac_e  
0                         0.003725    0.750587       -10.0  6.993838  

[1 rows x 913 columns]

In [27]:
pd.concat([
    measurements.loc['[6]Glc'].to_frame().T.iloc[:, :9],
    measurements.loc['BOM'].to_frame().T
], axis=1)

data_id,"ala__L_c|[1,2]_{M-}+0","ala__L_c|[1,2]_{M-}+1","ala__L_c|[1,2]_{M-}+2",ala__L_c_{M-}+0,ala__L_c_{M-}+1,ala__L_c_{M-}+2,ala__L_c_{M-}+3,gly_c|[1]_{M-}+0,gly_c|[1]_{M-}+1,biomass_rxn,EX_glc__D_e,EX_ac_e
0,0.464283,0.528698,0.007019,0.466426,0.523201,0.008477,0.001896,0.976139,0.023861,0.750587,-10.0,6.993838


Note that in the cell above, one of the experiments is called `BOM`, which is an abbreviation for boundary observation model (BOM). These **DO NOT** correspond to the boundary fluxes that were measured by Leighty, which are reported in the `1-s2.0-S1096717615000038-mmc4.xlsx` file included in the `sbmfi.models` folder. Instead, **we chose to parse the inferred fluxes** from the `1-s2.0-S1096717615000038-mmc1.xlsx` file and use the inferred boundary fluxes as pseudo-measurements. 

Flux parsing was non-trivial, since the BiGG models have some reactions that do not occur in the Antoniewicz model, e.g. the balancing of water and some co-factors that were left out of the Antoniewicz model. Furthermore, Crown reports the following flux coordinates: $v^{net}$ according to the normal definition and a quantity $v^{diff} = v^{fwd} - v^{rev}$. Last, some reactions are defined in opposite directions compared to the BiGG models, and this is what the `coef` keyword in the `anton_reaction_kwargs` dictionary is used for. For the parsing, I defined some fluxes to be free (i.e. rref basis kernel) and subsequently balancing the dependent fluxes from this basis. I subsequently mapped fluxes to the thermodynamic coordinate system defined as follows:

\begin{align}
    v_r^{\, net} & = v_r^{\, fwd} - v_r^{\, rev} & \text{net flux}\\
    \rho_r & = \frac{v_r^{\, rev}}{v_r^{\, fwd}} & \text{flux fraction}\\ 
    v_r^{\, xch} & = \text{pow}(\rho_r, \text{sgn} v_r^{\, net}) & \text{exchange flux}
\end{align}

Here the vectors $v_r^{\, fwd}$ and $v_r^{\, rev}$ are the fluxes in the labelling coordinate system

The function `sbmfi.models.build_models._parse_anton_fluxes` is used to do the parsing and the parsed result is stored in `parsed_anton_fluxes.csv`. For quick access, the function `sbmfi.models.build_models.read_anton_fluxes` reads the file and extracts the `fluxes_df`.

In [28]:
# these are the MLE inferred fluxes from the Crown publication in the thermodynamic coordinate system
anton_kwargs['fluxes_df'].iloc[-5:]

,NADH16,CYTBD,ATPS4r,THD2,ATPM,ADK1,O2t,EX_o2_e,NH4t,EX_nh4_e,...,ACONTb_xch,ICDHyr_xch,SUCOAS_xch,SUCDi_xch,FUM_xch,MDH_xch,PTAr_xch,ACKr_xch,GHMT2r_xch,GLYCL_xch
[3]Glc,22.184933,22.193022,20.911112,8.065342,5.278135,0.812158,11.096511,-11.096511,5.788182,-5.788182,...,0.344481,0.918404,0.001,0.999000,0.995009,0.999000,0.636620,0.999,0.964599,0.00100
[4]Glc,46.548276,48.439035,64.927356,-15.077670,60.262977,0.506889,24.219518,-24.219518,3.612558,-3.612558,...,0.911806,0.916647,0.999,0.873784,0.956981,0.957785,0.711139,0.999,0.952471,0.99834
[5]Glc,26.348326,27.188859,27.776600,5.974424,15.744217,0.754095,13.594430,-13.594430,5.374371,-5.374371,...,0.831166,0.001000,0.999,0.981904,0.987828,0.654139,0.119689,0.001,0.044458,0.00100
[6]Glc,27.464780,28.361999,29.799218,4.870716,17.834983,0.740462,14.180999,-14.180999,5.277216,-5.277216,...,0.584454,0.883879,0.999,0.331906,0.969587,0.965740,0.547887,0.999,0.001000,0.30270
COMPLETE-MFA,27.469312,28.560956,29.664041,5.384075,18.336062,0.735044,14.280478,-14.280478,5.238596,-5.238596,...,0.199210,0.638363,0.001,0.982574,0.848850,0.999000,0.632023,0.999,0.470994,0.00100


### Building the dataset simulator

In the small model in the first section of this notebook, we manually picked 4 flux values and simulated those. In any machine learning scenario, we need to sample fluxes according to some prior distribution over the polytope and then simulate measurements given a set of substrate labellings defined in `substrate_df`. We will first introduce the prior and then look at defining an observation model for MDVs measured by mass-spec. Last, we will combine observation models with 


#### Priors

In this version of `sbmfi` we included the `UniFluxPrior` which samples from a uniform density over net fluxes. The arguments to the `UniFluxPrior` are:
- `model`: either a `LabellingModel` or a `FluxCoordinateMapper`
- `xch_prior`: a subclass of `_XchFluxPrior`; if `None` is passed, we automatically construct a `UniXchFluxPrior` which samples from a uniform density over exchange fluxes
- `cache_size`: sampling densities in a polytope is typically done using some hit-and-run algorithm. It is inefficient to draw a small number of samples, and thus internally we draw `cache_size` samples from the polytope and store them in a cache. The requested number of draws is then returned from the cache. Currently it is not possible to request `n > cache_size`, since I never needed it, though this functionality could be added.

Priors are `torch.distributions.distribution.Distribution` objects and implement the following functionality: sampling using the `.sample` function, evaluation using the `log_prob` function and checking the support (in this case the polytope) using the `prior.support` attribute. Below we show how to sample and evaluate probability; for now the probatility returns 0-s since the log--probability of the uniform prior is $p(\vec{y}) = \frac{1}{\text{vol}\mathcal{F}^t}$ which is the volume of the net-flux polytope. This volume equals the volume of the net-flux polytope plus the volume of the exchange flux hyper-rectangle. Since $p(\vec{y})$ is constant given a particular polytope, it is a constant and can thus be ignored for most inference scenarios over that polytope. 

In [29]:
from sbmfi.inference.priors import UniFluxPrior
from sbmfi.core.observation import ClassicalObservationModel

In [30]:
up = UniFluxPrior(
    model=anton_model,
    xch_prior = None, # allows to specify different exchange flux priors
    cache_size = 20000,
)
UniFluxPrior.__mro__  # note that the prior is a torch.distributions.distribution.Distribution object!

(sbmfi.inference.priors.UniFluxPrior,
 sbmfi.inference.priors._NetFluxPrior,
 sbmfi.inference.priors._BasePrior,
 torch.distributions.distribution.Distribution,
 object)

In [31]:
draws = up.sample((2,)) # regular sampling returns a torch.Tensor
draws

tensor([[-0.2704,  0.7816,  1.0432, -1.3773,  0.9931,  0.1895, -0.9151,  0.1489,
          0.3630,  0.2888,  0.5913,  0.6617,  0.5835,  0.4248,  0.3988,  0.9667,
          0.6137,  0.1786,  0.2078,  0.2977,  0.6658,  0.5793,  0.4014,  0.1309,
          0.9531,  0.9415,  0.0315,  0.4207,  0.8578,  0.4777,  0.5698,  0.5755,
          0.9067,  0.3700,  0.9712],
        [ 0.0979,  0.3453,  0.5507, -0.6546,  1.1565, -1.4755,  0.4714,  0.1924,
          0.5812, -1.4208,  0.7239,  0.2388,  0.9507,  0.7516,  0.8322,  0.5128,
          0.0122,  0.2884,  0.6849,  0.0152,  0.3808,  0.3949,  0.0493,  0.7083,
          0.7745,  0.1014,  0.6281,  0.5970,  0.9934,  0.4035,  0.9853,  0.4342,
          0.1984,  0.6084,  0.4288]])

In [32]:
up.log_prob(draws)

tensor([[0.],
        [0.]])

In [33]:
up.sample_pandalize(2) # sample_pandalize returns a dataframe

theta_id,B_svd0,B_svd1,B_svd2,B_svd3,B_svd4,B_svd5,B_svd6,B_svd7,B_svd8,B_svd9,...,ACONTb_xch,ICDHyr_xch,SUCOAS_xch,SUCDi_xch,FUM_xch,MDH_xch,PTAr_xch,ACKr_xch,GHMT2r_xch,GLYCL_xch
0,-0.204504,1.339512,-0.279243,-0.191774,0.220227,-0.634345,-0.359284,-0.414722,-0.096300,1.507079,...,0.483932,0.677111,0.457861,0.428569,0.491106,0.310065,0.014170,0.726797,0.713760,0.743978
1,0.270833,-1.132824,0.218532,-1.613348,-0.126183,0.349735,1.396702,-0.861333,-0.092431,0.796057,...,0.971554,0.785823,0.552011,0.258830,0.344626,0.878223,0.347846,0.455172,0.957819,0.173742


#### Defining an MDV observation model

To simulate measurements, we need to link things we can measure (defined in the `annotation_df`) to things we can simulate and additionally, we need to simulate measurement errors. This is the purpose of an observation model. The classical observation model is defined as follows: $\vec{x} \sim \mathcal{N}(\vec{\mu}, \pmb{\Sigma})$, where $\vec{x}$ are noise-corrupted measurements, $\vec{\mu}$ is a vector of partial MDVs and $\pmb{\Sigma}$ is a covariance matrix. To explain partial MDVs, lets look at acetate with the following MDV: $\vec{s}^{ac} = [\frac{c_{M+0}^{ac}}{\vec{1}^T \cdot \vec{c}^{ac}}, \frac{c_{M+1}^{ac}}{\vec{1}^T \cdot \vec{c}^{ac}}, \frac{c_{M+2}^{ac}}{\vec{1}^T \cdot \vec{c}^{ac}}]^T$, where $\vec{c}$ represents the concentrations mass-isotopomers. A common scenario is that we cannot measure some mass-isotopomer signal (e.g. $c_{M+2}^{ac}$). This could be because it falls under the limit of detection of our instrument or because its signal is spectrally irresolvable from another signal with the same mass and elution time. In this case, there should be no entry for $c_{M+2}^{ac}$ in the `annotation_df`. The partial MDV of acetate is then $\vec{\mu} = [\frac{c_{M+0}^{ac}}{c_{M+0}^{ac}+c_{M+1}^{ac}},\frac{c_{M+1}^{ac}}{c_{M+0}^{ac}+c_{M+1}^{ac}}]^T$. Note that partial MDVs are different from the MDV of a metabolite fragment (as obtained by for instance MS-MS). Metabolite fragments can be added to `annotation_df` with the following notation: `ac_c|[0,1]`, which means that we measure a fragment of a metabolite with identifier $ac_c$ (acetate) that contains the zeroth and first carbon atoms.

In [34]:
annotation_df.iloc[:3,:] # metabolite fracment notation

,met_id,nC13,adduct_name,formula,sigma
0,"ala__L_c|[1,2]",0,M-,C10H26ONSi2,0.01
1,"ala__L_c|[1,2]",1,M-,C10H26ONSi2,0.01
2,"ala__L_c|[1,2]",2,M-,C10H26ONSi2,0.01


For the Leighty model, we assume a measurement measurement variance $\sigma^2 = 0.01$ and we have indicated this is the `sigma` column in `annotation_df`, which is not a required columns, but often useful. There are some convenience functions to churn the `sigma` column into covariance matrix $\pmb{\Sigma}$. A first step is to generate an `observation_df` which is what is used internally in the `sbmfi.core.observation.MDV_ObservationModel` which is the baseclass for all observation models for MDVs. The `observation_df` is used internally to map all sorts of things to each other and it has the following columns:

- `observation_id`: unique data identifiers, this is what we end up simulating!
- `annot_df_idx`: the index of this observation in the the `annotation_df`
- `met_id`: BiGG metabolite identifier, possibly with specification of fragment
- `ion_id`: combination of `met_id` and adduct
- `formula`, `adduct_name`, `nC13`: self-explanatory or see above
- `isotope_decomposition` : chemical formula with isotopes; all entries without isotope annotation are the most abundant isotope of that element.
- `state_idx`: index of the simulated MDV element (from `model.cascade`) is in the numerator of the partial MDV
- `sigma`: sigma from the `annotation_df`

Note that in case one has repeated measurements of a sample, one could construct the covariance matrix manually and pass that instead.

In [35]:
observation_df = ClassicalObservationModel.generate_observation_df(anton_model, annotation_df)
observation_df.iloc[:7]

,annot_df_idx,met_id,ion_id,formula,adduct_name,nC13,isotope_decomposition,state_idx,sigma
observation_id,,,,,,,,,
ala__L_c_{M-}+0,3,ala__L_c,ala__L_c_{M-},C11H26O2NSi2,M-,0,[12]C11H26NO2Si2-,3,0.01
ala__L_c_{M-}+1,4,ala__L_c,ala__L_c_{M-},C11H26O2NSi2,M-,1,[12]C10[13]CH26NO2Si2-,4,0.01
ala__L_c_{M-}+2,5,ala__L_c,ala__L_c_{M-},C11H26O2NSi2,M-,2,[12]C9[13]C2H26NO2Si2-,5,0.01
ala__L_c_{M-}+3,6,ala__L_c,ala__L_c_{M-},C11H26O2NSi2,M-,3,[12]C8[13]C3H26NO2Si2-,6,0.01
"ala__L_c|[1,2]_{M-}+0",0,"ala__L_c|[1,2]","ala__L_c|[1,2]_{M-}",C10H26ONSi2,M-,0,[12]C10H26NOSi2-,0,0.01
"ala__L_c|[1,2]_{M-}+1",1,"ala__L_c|[1,2]","ala__L_c|[1,2]_{M-}",C10H26ONSi2,M-,1,[12]C9[13]CH26NOSi2-,1,0.01
"ala__L_c|[1,2]_{M-}+2",2,"ala__L_c|[1,2]","ala__L_c|[1,2]_{M-}",C10H26ONSi2,M-,2,[12]C8[13]C2H26NOSi2-,2,0.01


The `sigma` column of the `observation_df` specifies the diagonal of the covariance matrix:

In [36]:
diagonal = observation_df['sigma']**2
diagonal

observation_id
ala__L_c_{M-}+0              0.0001
ala__L_c_{M-}+1              0.0001
ala__L_c_{M-}+2              0.0001
ala__L_c_{M-}+3              0.0001
ala__L_c|[1,2]_{M-}+0        0.0001
                              ...  
val__L_c|[1,2,3,4]_{M-}+0    0.0001
val__L_c|[1,2,3,4]_{M-}+1    0.0001
val__L_c|[1,2,3,4]_{M-}+2    0.0001
val__L_c|[1,2,3,4]_{M-}+3    0.0001
val__L_c|[1,2,3,4]_{M-}+4    0.0001
Name: sigma, Length: 65, dtype: float64

We can use this diagonal to construct a covariance matrix as follows:

In [37]:
from sbmfi.core.util import _cell_color

In [38]:
sigma_x = ClassicalObservationModel.construct_sigma_x(observation_df, diagonal=diagonal, corr=0.0)
sigma_x.iloc[:7,:7].style.applymap(lambda x: _cell_color(x))

observation_id,ala__L_c_{M-}+0,ala__L_c_{M-}+1,ala__L_c_{M-}+2,ala__L_c_{M-}+3,"ala__L_c|[1,2]_{M-}+0","ala__L_c|[1,2]_{M-}+1","ala__L_c|[1,2]_{M-}+2"
observation_id,,,,,,,
ala__L_c_{M-}+0,0.000100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ala__L_c_{M-}+1,0.000000,0.000100,0.000000,0.000000,0.000000,0.000000,0.000000
ala__L_c_{M-}+2,0.000000,0.000000,0.000100,0.000000,0.000000,0.000000,0.000000
ala__L_c_{M-}+3,0.000000,0.000000,0.000000,0.000100,0.000000,0.000000,0.000000
"ala__L_c|[1,2]_{M-}+0",0.000000,0.000000,0.000000,0.000000,0.000100,0.000000,0.000000
"ala__L_c|[1,2]_{M-}+1",0.000000,0.000000,0.000000,0.000000,0.000000,0.000100,0.000000
"ala__L_c|[1,2]_{M-}+2",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000100


Now we can finally construct our MDV observation model. The arguments to the constructor are the following:

- `model`, `annotation_df`, `sigma_df`: self-explanatory; `sigma_df` is not a required argument, and if `None` is passed, we assume a diagonal covariance matrix with $\sigma_{ii}^2=0.01$.
- `omega`: this is the scaling factor introduced in one of the original [cumomer papers](https://onlinelibrary.wiley.com/doi/10.1002/(SICI)1097-0290(1999)66:2%3C86::AID-BIT2%3E3.0.CO;2-A)
- `correct_natab`: whether to correct the resulting partial MDVs for natural abundance; **TODO**: I have not looked into this since quite some time, so not sure it currently works. I think instead of correcting simulations, it is better to correct measurements for natural abundance using the strategy detailed above, since this saves computational effort. 
- `clip_min`: when simulating measurement errors, it is possible that $x = \mathcal{N}(\mu, \sigma)$ ends up below 0 and therefore `clip_min` sets negative values to 0.
- `normalize`: whether to normalize the sum of the simulated measurements of metabolite $m$ to 1: $\vec{1}^T \cdot \vec{x}^m = 1$


In [39]:
com = ClassicalObservationModel(
    model=anton_model,
    annotation_df=annotation_df,
    sigma_df = sigma_x,
    omega = None,
    correct_natab=False,
    clip_min=0.0,
    normalize=True,
)

The simulation of measurements thus consists of three steps:

1. map independent variables sampled from the prior to fluxes
2. simulate MDVs using a `LabellingModel`
3. simulate measurements using the `MDV_ObservationModel`



In [40]:
fcm = anton_model.flux_coordinate_mapper  # this is the flux coordinate mapper
labelling_fluxes = fcm.map_theta_2_fluxes(draws[:2], pandalize=True)  # map draws from the prior to fluxes

labelling_fluxes

,NADH16,CYTBD,ATPS4r,THD2,ATPM,ADK1,O2t,EX_o2_e,NH4t,EX_nh4_e,...,ACONTb_rev,ICDHyr_rev,SUCOAS_rev,SUCDi_rev,FUM_rev,MDH_rev,PTAr_rev,ACKr_rev,GHMT2r_rev,GLYCL_rev
0,60.519855,69.532152,78.930411,0.760191,20.648556,0.245984,34.766076,-34.766076,1.753111,-1.753111,...,149.296835,0.057113,2.360930,54.360175,8.328559,10.273548,9.763303,77.159141,0.110057,8.094035
1,49.550651,57.026707,63.843588,0.435127,23.786481,0.469935,28.513354,-28.513354,3.349188,-3.349188,...,0.902591,10.309619,13.301008,1128.890903,5.176951,260.195032,4.143685,6.735276,0.101802,0.195966


Note that this dataframe contains a bunch of fluxes related to co-factors such as `NADH16` and `O2t`; these are important for flux-balancing, but do not affect labelling dynamics. Labelling fluxes are thus a subset of the fluxes above; trimming non-labelling fluxes is done automatically by setting `trim=True` in the function below. Next we can simulate the full MDVs of all measured metabolites by calling `cascade` as in the small model above.

In [41]:
anton_model.set_fluxes(labelling_fluxes, trim=True)  # set fluxes and trim non-labelling fluxes
mdvs = anton_model.cascade(pandalize=True)
mdvs

mdv_id,"ala__L_c|[1,2]+0","ala__L_c|[1,2]+1","ala__L_c|[1,2]+2",ala__L_c+0,ala__L_c+1,ala__L_c+2,ala__L_c+3,gly_c|[1]+0,gly_c|[1]+1,gly_c+0,...,"glu__L_c|[1,2,3,4]+4",glu__L_c+0,glu__L_c+1,glu__L_c+2,glu__L_c+3,glu__L_c+4,glu__L_c+5,"tyr__L_c|[0,1]+0","tyr__L_c|[0,1]+1","tyr__L_c|[0,1]+2"
0,0.417153,0.282284,0.300563,0.381968,0.187257,0.380203,0.050572,0.562724,0.437276,0.548162,...,0.098318,0.182387,0.213386,0.330506,0.165370,0.103855,0.004496,0.514788,0.404236,0.080976
1,0.438556,0.160970,0.400473,0.423467,0.108334,0.417499,0.050699,0.519062,0.480938,0.496207,...,0.149259,0.179177,0.146505,0.358650,0.147724,0.155902,0.012042,0.495088,0.413478,0.091434


We can now simulate `n_obs=2` measurements for each MDV vector from the observation model. Note that we call the `com` object, which internally uses the special `__call__` python dunder method to simulate measurements; this was convenient for all sorts of reasons which Ill not discuss here.

In [42]:
data = com(mdvs, n_obs=2, pandalize=True)
data

observation_id    ala__L_c_{M-}+0  ala__L_c_{M-}+1  ala__L_c_{M-}+2  \
samples_id obs_i                                                      
0          0             0.401170         0.199195         0.375542   
           1             0.420251         0.088836         0.421913   
1          0             0.376059         0.202395         0.365719   
           1             0.426857         0.103209         0.413920   

observation_id    ala__L_c_{M-}+3  ala__L_c|[1,2]_{M-}+0  \
samples_id obs_i                                           
0          0             0.024093               0.414250   
           1             0.069001               0.439615   
1          0             0.055827               0.428781   
           1             0.056014               0.450738   

observation_id    ala__L_c|[1,2]_{M-}+1  ala__L_c|[1,2]_{M-}+2  \
samples_id obs_i                                                 
0          0                   0.295448               0.290302   
           1                   0.152608               0.407777   
1          0                   0.265394               0.305825   
           1                   0.153429               0.395833   

observation_id    asp__L_c_{M-}+0  asp__L_c_{M-}+1  asp__L_c_{M-}+2  ...  \
samples_id obs_i                                                     ...   
0          0             0.410427         0.153254         0.380383  ...   
           1             0.394524         0.121903         0.421548  ...   
1          0             0.395375         0.167287         0.370749  ...   
           1             0.388940         0.114852         0.406052  ...   

observation_id    val__L_c_{M-}+1  val__L_c_{M-}+2  val__L_c_{M-}+3  \
samples_id obs_i                                                      
0          0             0.193652         0.325202         0.186088   
           1             0.113943         0.373830         0.131813   
1          0             0.198058         0.323526         0.175813   
           1             0.114894         0.374652         0.109759   

observation_id    val__L_c_{M-}+4  val__L_c_{M-}+5  val__L_c|[1,2,3,4]_{M-}+0  \
samples_id obs_i                                                                
0          0             0.128213         0.001941                   0.167279   
           1             0.165667         0.013884                   0.185047   
1          0             0.136957         0.014455                   0.162714   
           1             0.187848         0.030865                   0.190299   

observation_id    val__L_c|[1,2,3,4]_{M-}+1  val__L_c|[1,2,3,4]_{M-}+2  \
samples_id obs_i                                                         
0          0                       0.224396                   0.351614   
           1                       0.160204                   0.364892   
1          0                       0.225652                   0.334389   
           1                       0.151154                   0.378837   

observation_id    val__L_c|[1,2,3,4]_{M-}+3  val__L_c|[1,2,3,4]_{M-}+4  
samples_id obs_i                                                        
0          0                       0.161171                   0.095540  
           1                       0.124986                   0.164871  
1          0                       0.176816                   0.100429  
           1                       0.133042                   0.146668  

[4 rows x 65 columns]

#### Boundary flux observation model

In $^{13}$C-MFA, it is also common to measure boundary fluxes such as growth rate, substrate uptake and excretion of metabolites, such as those resulting from overflow metabolism (e.g. acetate). To simulate those measurements, we have a `sbmfi.core.observation.BoundaryObsevationModel` base class. It is common to assume that boundary flux measurements are also drawn from a multivariate normal: $\vec{x}^{bd} \sim \mathcal{N}(\vec{\mu}^{bd}, \pmb{\Sigma})$, where $\vec{\mu}^{bd}$ is determined by draws from the prior. In `sbmfi` we have the `MVN_BoundaryObservationModel` which takes the following constructor arguments:
- `fcm`: a FluxCoordinateMapper object
- `measured_boundary_fluxes`: the measured boundary fluxes, these are specified as the identifiers of net-fluxes (NOT LABELLING FLUXES)
- `biomass_id`: the identifier of the biomass flux
- `sigma_o`: similar to `sigma_df`, covariance matrix of the MVN
- `biomass_var`: in case `sigma_o=None` is passed, this is the variance that we assume for the growth rate measurement
- `boundary_var`: in case `sigma_o=None` is passed, this is the variance that we assume for the boundary fluxes

If we do multiple labelling experiments including boundary flux measurements, it is easy to compute `sigma_o`, since we can compute it from the measurements of boundary fluxes across experiments.

In [43]:
from sbmfi.core.observation import MVN_BoundaryObservationModel
measured_boundary_fluxes = anton_kwargs['measured_boundary_fluxes']
bm_id = anton_kwargs['bm_id']
print(f"biomass id in the anton model: {bm_id}, \nmeasured boundary fluxes: {measured_boundary_fluxes}")
bom = MVN_BoundaryObservationModel(
    fcm=fcm,
    measured_boundary_fluxes=anton_kwargs['measured_boundary_fluxes'],
    biomass_id=bm_id,
    sigma_o=None,
    biomass_var=0.01,
    boundary_var=0.2,
)

biomass id in the anton model: biomass_rxn, 
measured boundary fluxes: ['biomass_rxn', 'EX_glc__D_e', 'EX_ac_e']


#### Constructing the data set simulator

The last step in simulating a data-set is to combine one `MDV_ObservationModel` per labelling experiment with a `BoundaryObsevationModel`, if boundary fluxes are measured, which is not a hard requirement. In the example below, we want to simulate data for two labelling experiments:
- `[1]Glc`: 100% 1-labelled glucose 
- `20% [U]Glc`: 20% U-glucose : 80% natural-glucose

To do so, we select those labelling conditons from the `substrate_df` and construct a dictionary, where the `substrate_id` is the key and the corresponding observation model is the value. In this case we use the same `annotation_df` for both observation models, but in reality these might be different per experiment!

The constructor arguments to the `DataSetSim` are:
- `model`, `boundary_observation_model`, `substrate_df`: previously detailed
- `mdv_observation_models`: dictionary with keys being experiment identifiers `substrate_id` and values being observation models
- `num_processes`: number of processes to use for python multiprocessing
- `epsilon`: this is a tolerance to filter faulty simulations (metabolite MDVs should sim to 1+-epsilon)


In [44]:
from sbmfi.core.simulator import DataSetSim
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

experiment_labellings = substrate_df.loc[['20% [U]Glc', '[1]Glc']]
obmods = {}
for substrate_id, row in experiment_labellings.iterrows():
    print(f'substrate_id: {substrate_id}, (part of the) substrate labelling vector:')
    print(row[row>0].to_frame().T.iloc[:, :8])
    print()
    obmods[substrate_id] = ClassicalObservationModel(
        model=anton_model,
        annotation_df=annotation_df,  # IN REALITY THIS SHOULD BE UNIQUE TO AN EXPERIMENT!
        sigma_df = sigma_x, # IN REALITY THIS SHOULD BE UNIQUE TO AN EXPERIMENT!
        omega = None,
        correct_natab=False,
        clip_min=0.0,
        normalize=True,
    )

dss = DataSetSim(
    model=anton_model,
    substrate_df=substrate_df,
    mdv_observation_models=obmods,
    boundary_observation_model=bom,  # hence we do measure boundary fluxes
    num_processes=2,
    epsilon=1e-12,
)

print(dss.data_id)  # here we see that the data is made up of MDVs per experiment and boundary fluxes

substrate_id: 20% [U]Glc, (part of the) substrate labelling vector:
            glc__D_e/000000  glc__D_e/000001  glc__D_e/100000  glc__D_e/010000  glc__D_e/001000  glc__D_e/000100  glc__D_e/000010  glc__D_e/100001
20% [U]Glc         0.745333         0.008061         0.008061         0.008061         0.008061         0.008061         0.008061         0.000087

substrate_id: [1]Glc, (part of the) substrate labelling vector:
        glc__D_e/000000  glc__D_e/100000  co2_e/0  co2_e/1
[1]Glc            0.004            0.996    0.989    0.011

MultiIndex([('20% [U]Glc',           'ala__L_c_{M-}+0'),
            ('20% [U]Glc',           'ala__L_c_{M-}+1'),
            ('20% [U]Glc',           'ala__L_c_{M-}+2'),
            ('20% [U]Glc',           'ala__L_c_{M-}+3'),
            ('20% [U]Glc',     'ala__L_c|[1,2]_{M-}+0'),
            ('20% [U]Glc',     'ala__L_c|[1,2]_{M-}+1'),
            ('20% [U]Glc',     'ala__L_c|[1,2]_{M-}+2'),
            ('20% [U]Glc',           'asp__L_c_{M-}+0')

Lets now draw a "large" number of samples from the prior:

In [45]:
draws = up.sample((1000,))

There are three ways of simulating from the dss. The first function is just `dss.simulate` which takes the following arguments:

- `theta`: theta needs to be a 2D array with the following shape: `batch_size x n_theta`!
- `n_obs`: **this is a special parameter** which sets the number of data vectors that is simulated per flux sample from the observation model! If we sample 2 fluxes and sample 3 observations per flux, we end up with 6 simulated data points. Additionally, when `n_obs=0`, we do not sample any observation noise and just return the computed partial MDVs $\vec{\mu}$. For `n_obs>0` we sample measurement errors from the observation model
- `return_mdvs`: whether to return simulated measurements or full mdvs
- `pandalize`: whether to returen a dataframe

In [46]:
mdvs = dss.simulate(
    theta=draws[:2],
    n_obs=0,  # ignored since we return MDVs
    return_mdvs=True,
    pandalize=True,
)
mdvs

labelling_id       20% [U]Glc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               \
mdv_id       ala__L_c|[1,2]+0 ala__L_c|[1,2]+1 ala__L_c|[1,2]+2 ala__L_c+0 ala__L_c+1 ala__L_c+2 ala__L_c+3 gly_c|[1]+0 gly_c|[1]+1   gly_c+0   gly_c+1   gly_c+2 val__L_c|[1,2,3,4]+0 val__L_c|[1,2,3,4]+1 val__L_c|[1,2,3,4]+2 val__L_c|[1,2,3,4]+3 val__L_c|[1,2,3,4]+4 val__L_c+0 val__L_c+1 val__L_c+2 val__L_c+3 val__L_c+4 val__L_c+5 leu__L_c|[1,2,3,4,5]+0 leu__L_c|[1,2,3,4,5]+1 leu__L_c|[1,2,3,4,5]+2 leu__L_c|[1,2,3,4,5]+3 leu__L_c|[1,2,3,4,5]+4 leu__L_c|[1,2,3,4,5]+5 ile__L_c|[1,2,3,4,5]+0 ile__L_c|[1,2,3,4,5]+1 ile__L_c|[1,2,3,4,5]+2 ile__L_c|[1,2,3,4,5]+3 ile__L_c|[1,2,3,4,5]+4 ile__L_c|[1,2,3,4,5]+5 ser__L_c|[1,2]+0 ser__L_c|[1,2]+1 ser__L_c|[1,2]+2 ser__L_c+0 ser__L_c+1 ser__L_c+2 ser__L_c+3 phe__L_c|[0,1]+0 phe__L_c|[0,1]+1 phe__L_c|[0,1]+2 phe__L_c|[1,2,3,4,5,6,7,8]+0 phe__L_c|[1,2,3,4,5,6,7,8]+1 phe__L_c|[1,2,3,4,5,6,7,8]+2 phe__L_c|[1,2,3,4,5,6,7,8]+3 phe__L_c|[1,2,3,4,5,6,7,8]+4 phe__L_c|[1,2,3,4,5,6,7,8]+5 phe__L_c|[1,2,3,4,5,6,7,8]+6 phe__L_c|[1,2,3,4,5,6,7,8]+7   
0                    0.733079         0.113894         0.153027   0.714096   0.093292   0.051360   0.141251    0.790329    0.209671  0.754904  0.109943  0.135153             0.537405             0.166987             0.237333             0.034858             0.023417   0.523489   0.149722   0.157552   0.123674   0.023947   0.021615               0.425010               0.244457               0.222621               0.077204                0.02581               0.004898               0.469257               0.214992               0.221343               0.065824               0.024322               0.004263         0.638291         0.303923         0.057786   0.611863   0.225735   0.119323   0.043079         0.770463         0.044427         0.185110                     0.431300                     0.102884                     0.207388                     0.036921                     0.133618                     0.025736                     0.051183                     0.005227   
1                    0.712780         0.155079         0.132141   0.679464   0.136985   0.067683   0.115868    0.790217    0.209783  0.752220  0.097636  0.150145             0.508055             0.221074             0.212425             0.040985             0.017461   0.484308   0.203010   0.159272   0.111186   0.026912   0.015311               0.402049               0.280953               0.214230               0.076756                0.02237               0.003643               0.441350               0.265797               0.207056               0.064044               0.019276               0.002477         0.645814         0.288497         0.065689   0.616666   0.208079   0.122737   0.052518         0.750834         0.085564         0.163601                     0.383971                     0.164656                     0.184687                     0.068987                     0.110479                     0.039592                     0.037627                     0.006553   

labelling_id                           

The `dss.simulate` function is useful to quickly simulate from small batches of fluxes; internally, we loop over all substrate labellings, compute the MDVs and parse everything into the correct shapes. The looping over substrate labellings means we need to internally reset some matrices (though they are cached) and this is not optimal, therefore to simulate larger batches of fluxes, we can use `dss.simulate_set` with the following arguments:

- `theta`, `n_obs`: same as above, except that `theta` can have arbitrary (large) sizes
- `fluxes_per_task`: manual setting of fluxes per task that is sent to a worker in a multiprocessing context; if `None`, we divide the total number of fluxes across `n_processes` and across number of labelling experiments
- `what`: what to simulate, `what=='mdv'` returns MDVs, `what=='data'` returns data, and `what=='all'` returns both
- `close_pool`: whether to close the pool of workers after this simulation. When simulating large datasets, we inevitably run into memory issues. The way the software is designed, it is easiest to draw a large number of samples from a prior, simulate data in batches as large as memory allows and store the batches in an `.hdf` file, which we discuss below.
- `show_progress`: whether to show a progress bar

Simulate_set returns a `dict` with the simulation input and results:

In [47]:
result = dss.simulate_set(
    theta=draws,
    fluxes_per_task=50, # 
    n_obs=3,
    what='all',
    show_progress=True,
    close_pool=False,
)

100%|██████████████████████████████████████████████████████████| 2000/2000 [00:15<00:00, 130.79it/s]


In [48]:
print(f" The shape of MDVs is: {result['mdv'].shape}, first dimension is the number of fluxes, \
\n second the number of labelling experiments and third the length of the LabellingModel state vector \
\n \
\n The shape of data is: {result['data'].shape}, first dimension is number of fluxes,  \
\n second the observation model subsamples (n_obs) and third the lenght of the data vector len(dss.data_id)")

 The shape of MDVs is: torch.Size([1000, 2, 80]), first dimension is the number of fluxes, 
 second the number of labelling experiments and third the length of the LabellingModel state vector 
 
 The shape of data is: torch.Size([1000, 3, 133]), first dimension is number of fluxes,  
 second the observation model subsamples (n_obs) and third the lenght of the data vector len(dss.data_id)


The last way to simulate data is to call the object `dss(...)`, which internally which again internally uses the special `__call__` python dunder method. This method takes `theta` instead of `fluxes` and returns only `data` instead of a `dict`. This method takes arguments:

- `theta`: arbitrarlily batched parameters (e.g. from prior), these are internally mapped to fluxes which are then passed to `dss.simulate_set`
- `n_obs`, `fluxes_per_task`, `close_pool`, `show_progress`: same as in `dss.simulate_set`
- `pandalize`: whether to return a dataframe

This way of simulating makes some things in machine learning easier but is irrelevant for simulating larger datasets, which is preferably done via `dss.simulate_set`.

In [49]:
dape = draws.shape
batched_draws = draws.view((10,10,10, dape[-1]))
print(batched_draws.shape)
batched_data = dss(batched_draws, n_obs=3)
print(batched_data.shape)

torch.Size([10, 10, 10, 35])
torch.Size([10, 10, 10, 3, 133])


Now that we have simulated some larger batch of data, it is useful to store it somehow. In principle a user can do this manually, but I have some rudimentary functions to store results to a `.h5` file and to read them back. `dss.to_hdf` takes the following arguments:

- `hdf`: either an open `pytables.File` or a string path to a file 
- `result`: dictionary with the results of `dss.simulate_set`
- `dataset_id`: a dataset identifier, in case we want to store different but similar simulations in one big file (simulations from prior and some posterior)
- `append`: whether to append simulations to an existing dataset or to destroy a dataset of the same name if it exists and make a new one with the entries in `result`
- `expectedrows_multiplier`: this is a `pytables` argument that indicates how many multiples of the current data-set we can expect to append to the dataset; please read the pytables documentation for more explanation.

In [50]:
result_1 = {k: v[:500] for k,v in result.items()}
result_1['theta'] = draws[:500]
result_2 = {k: v[500:] for k,v in result.items()}

dss.to_hdf(
    hdf='dss_dataset.h5', 
    dataset_id='prior_sampled',
    result=result_1,
    append=False, 
)

In [51]:
dss.to_hdf(  # trying to add the second dataset to the first, but we 'forgot' to add the prior draws
    hdf='dss_dataset.h5', 
    dataset_id='prior_sampled',
    result=result_2,
    append=True, 
)

In [52]:
result_2['theta'] = draws[500:]
dss.to_hdf(
    hdf='dss_dataset.h5', 
    dataset_id='prior_sampled',
    result=result_2,
    append=True, 
)

In [53]:
import tables as pt

hdf = pt.open_file('dss_dataset.h5')
print(hdf) 
print('NOTE that both result_1 and result_2 ended up in the right dataset in the file!')
hdf.close()

dss_dataset.h5 (File) ''
Last modif.: '2023-10-07T16:37:42+00:00'
Object Tree: 
/ (RootGroup) ''
/fluxes_id (Array(116,)) ''
/mdv_id (Array(80,)) ''
/theta_id (Array(35,)) ''
/data_id (Group) ''
/data_id/table (Table(133,)) ''
/prior_sampled (Group) ''
/prior_sampled/data (EArray(1500, 3, 133)) ''
/prior_sampled/mdv (EArray(1500, 2, 80)) ''
/prior_sampled/theta (EArray(1500, 35)) ''
/prior_sampled/validx (EArray(1500, 2)) ''
/substrate_df (Group) ''
/substrate_df/table (Table(2,)) ''

NOTE that both result_1 and result_2 ended up in the right dataset in the file!


To now read data from `test.h5`, we can use the `dss.read_hdf`, which takes the following arguments:

- `hdf`, `dataset_id`: same as in `dss.to_hdf`
- `what`: what to return if it is present in the dataset
- `start`, `stop`, `step`: used to sub-select data
- `pandalize`: as always...

In [54]:
data = dss.read_hdf(
    hdf='dss_dataset.h5', 
    dataset_id='prior_sampled',
    what='data', 
    pandalize=True
)
data.head(6)

labelling_id          20% [U]Glc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                \
data_id          ala__L_c_{M-}+0 ala__L_c_{M-}+1 ala__L_c_{M-}+2 ala__L_c_{M-}+3 ala__L_c|[1,2]_{M-}+0 ala__L_c|[1,2]_{M-}+1 ala__L_c|[1,2]_{M-}+2 asp__L_c_{M-}+0 asp__L_c_{M-}+1 asp__L_c_{M-}+2 asp__L_c_{M-}+3 asp__L_c_{M-}+4 asp__L_c|[1,2,3]_{M-}+0 asp__L_c|[1,2,3]_{M-}+1 asp__L_c|[1,2,3]_{M-}+2 asp__L_c|[1,2,3]_{M-}+3 glu__L_c_{M-}+0 glu__L_c_{M-}+1 glu__L_c_{M-}+2 glu__L_c_{M-}+3 glu__L_c_{M-}+4 glu__L_c_{M-}+5 gly_c|[1]_{M-}+0 gly_c|[1]_{M-}+1 ile__L_c|[1,2,3,4,5]_{M-}+0 ile__L_c|[1,2,3,4,5]_{M-}+1 ile__L_c|[1,2,3,4,5]_{M-}+2 ile__L_c|[1,2,3,4,5]_{M-}+3 ile__L_c|[1,2,3,4,5]_{M-}+4 ile__L_c|[1,2,3,4,5]_{M-}+5 leu__L_c|[1,2,3,4,5]_{M-}+0 leu__L_c|[1,2,3,4,5]_{M-}+1 leu__L_c|[1,2,3,4,5]_{M-}+2 leu__L_c|[1,2,3,4,5]_{M-}+3 leu__L_c|[1,2,3,4,5]_{M-}+4 leu__L_c|[1,2,3,4,5]_{M-}+5 phe__L_c|[0,1]_{M-}+0 phe__L_c|[0,1]_{M-}+1 phe__L_c|[0,1]_{M-}+2 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+0 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+1 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+2 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+3   
samples_id i_obs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
0          0            0.725883        0.087968        0.044993        0.141156              0.728174              0.109934              0.161892        0.613068        0.141258        0.149604        0.077643        0.018427                0.623133                0.195606                0.143114                0.038147        0.460085        0.211078        0.218551        0.078503        0.031783        0.000000         0.801496         0.198504                    0.458630                    0.221033                    0.232475                    0.064429                    0.003320                    0.020113                    0.432749                    0.251556                    0.227438                    0.077607                    0.010650                    0.000000              0.764293              0.043307              0.192400                          0.395219                          0.109460                          0.212543                          0.053488   
           1            

To keep data and simulator organized, it is advisable to `pickle` the simulator with a name that is similar/same to the `.h5` file. Unpicling works and we can use the simulator again.

In [55]:
import pickle

pickle.dump(dss, open('dss.p', 'wb'))
dss_unpickled = pickle.load(open('dss.p', 'rb'))

dss_unpickled.simulate(
    theta=draws[:2],
    return_mdvs=False,
    pandalize=True,
)

Read LP format model from file C:\Users\SYSBCPU\AppData\Local\Temp\tmpwpe1oo0q.lp
Reading time = 0.01 seconds
: 112 rows, 207 columns, 1115 nonzeros


labelling_id          20% [U]Glc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                \
data_id          ala__L_c_{M-}+0 ala__L_c_{M-}+1 ala__L_c_{M-}+2 ala__L_c_{M-}+3 ala__L_c|[1,2]_{M-}+0 ala__L_c|[1,2]_{M-}+1 ala__L_c|[1,2]_{M-}+2 asp__L_c_{M-}+0 asp__L_c_{M-}+1 asp__L_c_{M-}+2 asp__L_c_{M-}+3 asp__L_c_{M-}+4 asp__L_c|[1,2,3]_{M-}+0 asp__L_c|[1,2,3]_{M-}+1 asp__L_c|[1,2,3]_{M-}+2 asp__L_c|[1,2,3]_{M-}+3 glu__L_c_{M-}+0 glu__L_c_{M-}+1 glu__L_c_{M-}+2 glu__L_c_{M-}+3 glu__L_c_{M-}+4 glu__L_c_{M-}+5 gly_c|[1]_{M-}+0 gly_c|[1]_{M-}+1 ile__L_c|[1,2,3,4,5]_{M-}+0 ile__L_c|[1,2,3,4,5]_{M-}+1 ile__L_c|[1,2,3,4,5]_{M-}+2 ile__L_c|[1,2,3,4,5]_{M-}+3 ile__L_c|[1,2,3,4,5]_{M-}+4 ile__L_c|[1,2,3,4,5]_{M-}+5 leu__L_c|[1,2,3,4,5]_{M-}+0 leu__L_c|[1,2,3,4,5]_{M-}+1 leu__L_c|[1,2,3,4,5]_{M-}+2 leu__L_c|[1,2,3,4,5]_{M-}+3 leu__L_c|[1,2,3,4,5]_{M-}+4 leu__L_c|[1,2,3,4,5]_{M-}+5 phe__L_c|[0,1]_{M-}+0 phe__L_c|[0,1]_{M-}+1 phe__L_c|[0,1]_{M-}+2 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+0 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+1 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+2 phe__L_c|[1,2,3,4,5,6,7,8]_{M-}+3   
samples_id obs_i                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
0          0            0.730838        0.091432        0.032078        0.145652              0.739852              0.110716              0.149432        0.583357        0.159200        0.147506        0.089155        0.020782                0.624031                0.199533                0.141770                0.034666        0.462875        0.212922        0.203936        0.081596        0.034048        0.004623         0.798481         0.201519                    0.456067                    0.217627                    0.214265                    0.063616                    0.036082                    0.012343                    0.447319                    0.246372                    0.215071                    0.070973                    0.014037                    0.006228              0.758805              0.050867              0.190328                          0.431422                          0.107403                          0.199318                          0.034246   
           1            

### Concluding remarks

The only thing that comes to mind that was not discussed in this tutorial yet was the specification of exchange flux bounds, which can be done using `sbmfi.core.reaction.LabellingReaction.rho_min = val` and `sbmfi.core.reaction.LabellingReaction.rho_max = val` to set the upper and lower bounds for exchange fluxes. These can also be defined in the `reaction_kwargs` disctionary where we also specify net flux bounds and atom transitions. In most scenarios, I would just leave them at the defaults `[0, 0.999]`.

Important things to consider when simulating a larger batch of fluxes:

- Be careful with the `batch_size` parameter of `sbmfi.core.linlg.LinAlg` since it determines the batched shape of many matrices used for the EMU simulation algorithm which all need to be stored in memory! I would recommend to set `batch_size = 50`, otherwise we compromise too much memory and cannot store larger `result` tensors anymore and there is hardly any gain in performance from going over 50
- It is possible that memory fills up for extremely large datasets, especially with higher `n_obs >> 1`, therefore we recommend playing around with batching the simulations and intermittently storing batches using `dss.to_hdf`

If you run into issues, please shoot me an email at **diederen@imsb.biol.ethz.ch**